In [128]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.linalg import toeplitz
from scipy.signal import convolve
from scipy.stats import pearsonr

In [129]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [190]:
# Condições iniciais do reservatório
Por = 0.35      # Porosidade
K_min = 38      # Módulo de volume da rocha (GPa)
mu_min = 44     # Módulo de cisalhamento da rocha (GPa)
Rho_min = 2.65  # Densidade do quartzo (g/cm^3)

# Condições iniciais dos fluidos do reservatório
filtrate_sat = np.array([0.0, 0.05, 0.2, 0.4, 0.6, 1.0])  # Fração de saturação do filtrado (água)
K_hc = 1.15   # Módulo de volume do óleo em GPa
K_filtrate = 3.0  # Módulo de volume da água em GPa
rho_hc = 0.8    # Densidade do óleo em g/cm^3
rho_mud = 1.4   # Densidade da água em g/cm^3

K_fluid_initial = K_hc

# Calculando a densidade inicial da camada c3
rho_c3_initial = Rho_min * (1 - Por) + Por * rho_hc
print("Densidade inicial da camada c3:", rho_c3_initial)

# # Calculando o módulo de volume inicial
# K_bulk_initial = ((K_min * (1 - Por) + K_fluid_initial * Por) +1/((1 - Por)/K_min + Por/ K_fluid_initial))/2
# mu_bulk_initial = mu_min * (1 - Por)

# # Calculando a velocidade de Vs da camada c3
# Vs_c3 = np.sqrt(mu_bulk_initial / rho_c3_initial)

# # Calculando a velocidade de Vp da camada c3
# Vp_c3 = np.sqrt((K_bulk_initial + (4 / 3) * mu_bulk_initial) / rho_c3_initial)

# print("Velocidade de Vs da camada c3:", Vs_c3)
# print("Velocidade de Vp da camada c3:", Vp_c3)



Densidade inicial da camada c3: 2.0025


In [191]:
# depth axis
z = np.arange(0, 1000, 1)
n_z = len(z)

# time axis
t = np.arange(0, 4.0, 0.004)
n_t = len(t)

# reservoir thikness
reservoir = 570

# vp values for each layer
vp_c1 = 2.8 # sandstone
vp_c2 = 4.4
vp_c3 = 2.5 # high porosity sandstone (reservoir)
vp_c4 = 5.0
vp_c5 = 6.5 # limestone

# vs values for each layer
vs_c1 = 1.5 # sandstone
vs_c2 = 2.2
vs_c3 = 1.5 # high porosity sandstone (reservoir)
vs_c4 = 3.5
vs_c5 = 4.20 # limestone


# rho values for each layer
rho_c1 = 2.1 # sandstone
rho_c2 = 2.4
rho_c3 = rho_c3_initial # high porosity sandstone (reservoir)
rho_c4 = 2.6
rho_c5 = 2.7 # limestone

# vp model
vp = np.zeros((n_z))
vp[0:250] = vp_c1
vp[250:500] = vp_c2
vp[500:reservoir] = vp_c3
vp[reservoir:750] = vp_c4
vp[750:1000] = vp_c5

# vs model
vs = np.zeros((n_z))
vs[0:250] = vs_c1
vs[250:500] = vs_c2
vs[500:reservoir] = vs_c3
vs[reservoir:750] = vs_c4
vs[750:1000] = vs_c5

# rho model
rho = np.zeros((n_z))
rho[0:250] = rho_c1
rho[250:500] = rho_c2
rho[500:reservoir] = rho_c3
rho[reservoir:750] = rho_c4
rho[750:1000] = rho_c5

In [192]:
# vp and vs logs plot
fig = make_subplots(rows=1, cols=3, subplot_titles=('Vp', 'Rho', 'Vs'))

fig.add_trace(go.Scatter(x=vp, y=z, mode='lines', name='Seismic trace',
                         line=dict(color='darkblue')), row=1, col=1)
fig.add_trace(go.Scatter(x=rho, y=z, mode='lines', name='Seismic trace',
                         line=dict(color='crimson')), row=1, col=2)
fig.add_trace(go.Scatter(x=vs, y=z, mode='lines', name='Seismic trace',
                         line=dict(color='crimson')), row=1, col=3)

# xaxis properties
fig.update_xaxes(title_text="(km/s)", row=1, col=1,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text="(g/cm³)", row=1, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text="(km/s)", row=1, col=3,
                 title_font=dict(size=22), tickfont=dict(size=20))

# yaxis properties
fig.update_yaxes(title_text="Time (s)", row=1, col=1, autorange="reversed",
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(row=1, col=2, autorange="reversed",
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(row=1, col=3, autorange="reversed",
                 title_font=dict(size=22), tickfont=dict(size=20))

fig.update_layout(height=500, width=900, showlegend=False, title_text='Vp, Rho and Vs logs',
                  title_font=dict(size=24))

fig.update_annotations(font=dict(size=24))

fig.show()

## <font color=cyan> **Construindo o dado de poço**

In [193]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

K_bulk_initial=rho_c3_initial*vp_c3**2-4/3*rho_c3_initial*vs_c3**2
K_fluid_initial=K_hc
#k_dry = K_min * (1 - Por)
#kdry = (k_s1 * ((phi*k0)/k_f1+1-phi)-k0) / ((phi*k0)/k_f1+(k_s1/k0)-1-phi)
K_dry = (K_bulk_initial * ((Por*K_min)/K_fluid_initial+1-Por)-K_min) / ((Por*K_min)/K_fluid_initial+(K_bulk_initial /K_min)-1-Por)
print (K_dry)
mu=rho_c3_initial*vs_c3**2


# Selecionando a camada do reservatório
# reservoir = 570  # Defina o valor apropriado para o limite superior do reservatório
Vp = vp[500:reservoir]   # Velocidade da onda P (m/s)
Vs = vs[500:reservoir]   # Velocidade da onda S (m/s)
Rho = rho[500:reservoir] # Densidade volumétrica (kg/m^3)

# Inicializando arrays para armazenar os valores
vp_values = np.zeros((len(Vp), len(filtrate_sat)))
vs_values = np.zeros((len(Vs), len(filtrate_sat)))
rho_values = np.zeros((len(Rho), len(filtrate_sat)))

# Listas para armazenar os valores intermediários
K_dry_list = []
K_fluid_list = []
K_sat_list = []
Rho_fluid_list = []
Rho_new_list = []
Vp_new_list = []
Vs_new_list = []

for i in range(len(filtrate_sat)):
    Swe = filtrate_sat[i]
    for j in range(len(Vp)):

        # Módulo de cisalhamento
       # mu = Rho[j] * (Vs[j]**2)

        # Módulo de volume da rocha seca
      #  K_dry = K_min * (1 - Por)
        # Módulo de volume do fluido misto (usando a fórmula correta)
        K_fluid =( 1 / ((Swe / K_filtrate) + ((1 - Swe) / K_hc))+(Swe * K_filtrate) + ((1 - Swe) * K_hc))/2

        # Densidade do fluido misto
        Rho_fluid = (1 - Swe)* rho_hc + Swe*rho_mud

        # Módulo de volume da rocha saturada
        K_sat = K_dry + ((1 - (K_dry / K_min))**2) / ((Por / K_fluid) + ((1 - Por) / K_min) + (K_dry / K_min**2))

        # Nova densidade
        Rho_new = (1 - Por) * Rho_min + Por * Rho_fluid

        # Nova velocidade de Vs
        Vs_new = np.sqrt(mu / Rho_new)

        # Nova velocidade de Vp
        Vp_new = np.sqrt((K_sat + (4 / 3) * mu) / Rho_new)

        # Atualizando os resultados para cada saturação
        vp_values[j, i] = Vp_new
        vs_values[j, i] = Vs_new
        rho_values[j, i] = Rho_new

        # Armazenando os valores nas listas
        K_dry_list.append(K_dry)
        K_fluid_list.append(K_fluid)
        K_sat_list.append(K_sat)
        Rho_fluid_list.append(Rho_fluid)
        Rho_new_list.append(Rho_new)
        Vp_new_list.append(Vp_new)
        Vs_new_list.append(Vs_new)
       # mu_new_list.append(mu)

vp_new_rsv = np.zeros((len(vp), len(filtrate_sat)))
vs_new_rsv = np.zeros((len(vs), len(filtrate_sat)))
rho_new_rsv = np.zeros((len(rho), len(filtrate_sat)))

for i in range(len(filtrate_sat)):
    vp_new_rsv[500:reservoir, i] = vp_values[:, i]  # Corrigido para atribuir a toda a extensão da faixa
    vs_new_rsv[500:reservoir, i] = vs_values[:, i]  # Corrigido para atribuir a toda a extensão da faixa
    rho_new_rsv[500:reservoir, i] = rho_values[:, i]  # Corrigido para atribuir a toda a extensão da faixa

# new vp log
vp_new_rsv[0:250, :] = vp_c1
vp_new_rsv[250:500, :] = vp_c2
vp_new_rsv[reservoir:750, :] = vp_c4
vp_new_rsv[750:1000, :] = vp_c5

# new vs log
vs_new_rsv[0:250, :] = vs_c1
vs_new_rsv[250:500, :] = vs_c2
vs_new_rsv[reservoir:750, :] = vs_c4
vs_new_rsv[750:1000, :] = vs_c5

# new rho log
rho_new_rsv[0:250, :] = rho_c1
rho_new_rsv[250:500, :] = rho_c2
rho_new_rsv[reservoir:750, :] = rho_c4
rho_new_rsv[750:1000, :] = rho_c5

# Convertendo as listas em arrays
K_dry_array = np.array(K_dry_list).reshape(len(filtrate_sat), len(Vp))
K_fluid_array = np.array(K_fluid_list).reshape(len(filtrate_sat), len(Vp))
K_sat_array = np.array(K_sat_list).reshape(len(filtrate_sat), len(Vp))
Rho_fluid_array = np.array(Rho_fluid_list).reshape(len(filtrate_sat), len(Vp))
Rho_new_array = np.array(Rho_new_list).reshape(len(filtrate_sat), len(Vp))
Vp_new_array = np.array(Vp_new_list).reshape(len(filtrate_sat), len(Vp))
Vs_new_array = np.array(Vs_new_list).reshape(len(filtrate_sat), len(Vp))

# Extraindo valores médios para cada saturação de filtrado
K_dry_values = np.mean(K_dry_array, axis=1)
K_final_sat_values = np.mean(K_sat_array, axis=1)
Rho_final_fluid_values = np.mean(Rho_fluid_array, axis=1)
Rho_final_values = np.mean(Rho_new_array, axis=1)
Vp_new_values = np.mean(Vp_new_array, axis=1)
Vs_new_values = np.mean(Vs_new_array, axis=1)

print(K_dry_values)
print(K_final_sat_values)
print(Rho_final_fluid_values)
print(Rho_final_values)
print(Vp_new_values)
print(Vs_new_values)

3.9954004001112877
[3.9954004 3.9954004 3.9954004 3.9954004 3.9954004 3.9954004]
[6.46523775 6.59491389 6.99380143 7.55822432 8.1832215  9.86018368]
[0.8  0.83 0.92 1.04 1.16 1.4 ]
[2.0025 2.013  2.0445 2.0865 2.1285 2.2125]
[2.49571295 2.50210184 2.52173738 2.5498363  2.58205454 2.67802809]
[1.5        1.49608281 1.48451286 1.46949572 1.45492529 1.42703913]


In [194]:
# Criando subplots
fig = make_subplots(rows=3, cols=2, subplot_titles=('K<sub>dry</sub> vs. saturation', 'K<sub>sat</sub> vs. saturation',
                                                    'Rho<sub>fl</sub> vs. saturation', 'Rho vs. saturation',
                                                    'V<sub>p</sub> vs. saturation', 'V<sub>s</sub> vs. saturation'))

# Adicionando subplots
fig.add_trace(go.Scatter(x=filtrate_sat, y=K_dry_values, mode='lines+markers', name='K_dry', marker=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=filtrate_sat, y=K_final_sat_values, mode='lines+markers', name='K_final_sat', marker=dict(color='red')), row=1, col=2)
fig.add_trace(go.Scatter(x=filtrate_sat, y=Rho_final_fluid_values, mode='lines+markers', name='Rho_fl', marker=dict(color='orange')), row=2, col=1)
fig.add_trace(go.Scatter(x=filtrate_sat, y=Rho_final_values, mode='lines+markers', name='Rho', marker=dict(color='orange')), row=2, col=2)
fig.add_trace(go.Scatter(x=filtrate_sat, y=Vp_new_values, mode='lines+markers', name='Vp', marker=dict(color='blue')), row=3, col=1)
fig.add_trace(go.Scatter(x=filtrate_sat, y=Vs_new_values, mode='lines+markers', name='Vs', marker=dict(color='blue')), row=3, col=2)

# Propriedades do eixo x
fig.update_xaxes(title_text='Filtrate saturation (v/v)', row=1, col=1, title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Filtrate saturation (v/v)', row=1, col=2, title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Filtrate saturation (v/v)', row=2, col=1, title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Filtrate saturation (v/v)', row=2, col=2, title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Filtrate saturation (v/v)', row=3, col=1, title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Filtrate saturation (v/v)', row=3, col=2, title_font=dict(size=22), tickfont=dict(size=20))

# Propriedades do eixo y
fig.update_yaxes(title_text='K<sub>dry</sub> (GPa)', row=1, col=1, title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(title_text='K<sub>sat</sub> (GPa)', row=1, col=2, title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(title_text='Rho<sub>fl</sub> (g/cm3)', row=2, col=1, title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(title_text='Rho (g/cm3)', row=2, col=2, title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(title_text='V<sub>p</sub> (km/s)', row=3, col=1, title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(title_text='V<sub>s</sub> (km/s)', row=3, col=2, title_font=dict(size=22), tickfont=dict(size=20))

# Layout da figura
fig.update_layout(height=800, width=1200, showlegend=False, title_text=f'Parameters - {len(vp[500:reservoir])}m of thickness and {round(Por*100)}% of porosity', title_font=dict(size=24), title_x=0.5)

fig.update_annotations(font=dict(size=24))

fig.show()

In [195]:
# # differents saturations plot
# fig = make_subplots(rows=1, cols=4, column_widths=[0.3, 0.3, 0.3, 0.1], subplot_titles=('Vp', 'Rho', 'Vs'))

# fig.add_trace(go.Scatter(x=vp_new_rsv[:,0], y=z, mode='lines', name='0% of invasion',
#                          line=dict(color='blue')), row=1, col=1)
# fig.add_trace(go.Scatter(x=vp_new_rsv[:,1], y=z, mode='lines', name='5% of invasion',
#                          line=dict(color='green')), row=1, col=1)
# fig.add_trace(go.Scatter(x=vp_new_rsv[:,2], y=z, mode='lines', name='20% of invasion',
#                          line=dict(color='violet')), row=1, col=1)
# fig.add_trace(go.Scatter(x=vp_new_rsv[:,3], y=z, mode='lines', name='40% of invasion',
#                          line=dict(color='red')), row=1, col=1)
# fig.add_trace(go.Scatter(x=vp_new_rsv[:,4], y=z, mode='lines', name='60% of invasion',
#                          line=dict(color='yellow')), row=1, col=1)
# fig.add_trace(go.Scatter(x=vp_new_rsv[:,5], y=z, mode='lines', name='100% of invasion',
#                          line=dict(color='black')), row=1, col=1)

# fig.add_trace(go.Scatter(x=rho_new_rsv[:,0], y=z, name='Sw=80%',
#                          line=dict(color='blue'), showlegend=False), row=1, col=2)
# fig.add_trace(go.Scatter(x=rho_new_rsv[:,1], y=z, name='Sw=60%',
#                          line=dict(color='green'), showlegend=False), row=1, col=2)
# fig.add_trace(go.Scatter(x=rho_new_rsv[:,2], y=z, name='Sw=40%',
#                          line=dict(color='violet'), showlegend=False), row=1, col=2)
# fig.add_trace(go.Scatter(x=rho_new_rsv[:,3], y=z, name='Sw=20%',
#                          line=dict(color='red'), showlegend=False), row=1, col=2)
# fig.add_trace(go.Scatter(x=rho_new_rsv[:,4], y=z, mode='lines', name='Sw=5%',
#                          line=dict(color='yellow'), showlegend=False), row=1, col=2)
# fig.add_trace(go.Scatter(x=rho_new_rsv[:,5], y=z, mode='lines', name='Sw=0%',
#                          line=dict(color='black'), showlegend=False), row=1, col=2)

# fig.add_trace(go.Scatter(x=vs_new_rsv[:,0], y=z, name='Sw=80%',
#                          line=dict(color='blue'), showlegend=False), row=1, col=3)
# fig.add_trace(go.Scatter(x=vs_new_rsv[:,1], y=z, name='Sw=80%',
#                          line=dict(color='green'), showlegend=False), row=1, col=3)
# fig.add_trace(go.Scatter(x=vs_new_rsv[:,2], y=z, name='Sw=80%',
#                          line=dict(color='violet'), showlegend=False), row=1, col=3)
# fig.add_trace(go.Scatter(x=vs_new_rsv[:,3], y=z, name='Sw=80%',
#                          line=dict(color='red'), showlegend=False), row=1, col=3)
# fig.add_trace(go.Scatter(x=vs_new_rsv[:,4], y=z, , name='Sw=5%',
#                          line=dict(color='yellow'), showlegend=False), row=1, col=3)
# fig.add_trace(go.Scatter(x=vs_new_rsv[:,5], y=z, , name='Sw=0%',
#                          line=dict(color='black'), showlegend=False), row=1, col=3)

# # xaxis properties
# fig.update_xaxes(title_text="(m/s)", row=1, col=1,
#                  title_font=dict(size=22), tickfont=dict(size=20))
# fig.update_xaxes(title_text="(kg/m³)", row=1, col=2,
#                  title_font=dict(size=22), tickfont=dict(size=20))
# fig.update_xaxes(title_text="(m/s)", row=1, col=3,
#                  title_font=dict(size=22), tickfont=dict(size=20))

# # yaxis properties
# fig.update_yaxes(title_text="Depth (m)", row=1, col=1, autorange="reversed",
#                  title_font=dict(size=22), tickfont=dict(size=20))
# fig.update_yaxes(autorange="reversed", row=1, col=2,
#                  title_font=dict(size=22), tickfont=dict(size=20))
# fig.update_yaxes(autorange="reversed", row=1, col=3,
#                  title_font=dict(size=22), tickfont=dict(size=20))

# fig.update_layout(height=600, showlegend=False, width=900, title_text='Saturations on reservoir',
#                   title_font=dict(size=24), title_x=0.5)

# fig.update_annotations(font=dict(size=24))

# # Colocando a legenda fora dos plots em um novo subplot
# fig.update_layout(
#     legend=dict(
#         x=1.1,
#         y=1,
#         traceorder='normal',
#         font=dict(
#             size=12,
#         ),
#     )
# )

# fig.show()

In [205]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Criação do subplot 2x3 (a última coluna para a legenda)
fig = make_subplots(rows=1, cols=4, column_widths=[0.3, 0.3, 0.3, 0.1], subplot_titles=('Vp', 'Rho', 'Vs', ''))

# Adicionando traços de Vp no subplot 1
fig.add_trace(go.Scatter(x=vp_new_rsv[:,0], y=z, mode='lines', name='0% of invasion',
                         line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=vp_new_rsv[:,1], y=z, mode='lines', name='5% of invasion',
                         line=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=vp_new_rsv[:,2], y=z, mode='lines', name='20% of invasion',
                         line=dict(color='violet')), row=1, col=1)
fig.add_trace(go.Scatter(x=vp_new_rsv[:,3], y=z, mode='lines', name='40% of invasion',
                         line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=vp_new_rsv[:,4], y=z, mode='lines', name='60% of invasion',
                         line=dict(color='yellow')), row=1, col=1)
fig.add_trace(go.Scatter(x=vp_new_rsv[:,5], y=z, mode='lines', name='100% of invasion',
                         line=dict(color='black')), row=1, col=1)

# Adicionando traços de Rho no subplot 2
fig.add_trace(go.Scatter(x=rho_new_rsv[:,0], y=z, name='Sw=80%',
                         line=dict(color='blue'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=rho_new_rsv[:,1], y=z, name='Sw=60%',
                         line=dict(color='green'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=rho_new_rsv[:,2], y=z, name='Sw=40%',
                         line=dict(color='violet'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=rho_new_rsv[:,3], y=z, name='Sw=20%',
                         line=dict(color='red'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=rho_new_rsv[:,4], y=z, mode='lines', name='Sw=5%',
                         line=dict(color='yellow'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=rho_new_rsv[:,5], y=z, mode='lines', name='Sw=0%',
                         line=dict(color='black'), showlegend=False), row=1, col=2)

# Adicionando traços de Vs no subplot 3
fig.add_trace(go.Scatter(x=vs_new_rsv[:,0], y=z, name='Sw=80%',
                         line=dict(color='blue'), showlegend=False), row=1, col=3)
fig.add_trace(go.Scatter(x=vs_new_rsv[:,1], y=z, name='Sw=60%',
                         line=dict(color='green'), showlegend=False), row=1, col=3)
fig.add_trace(go.Scatter(x=vs_new_rsv[:,2], y=z, name='Sw=40%',
                         line=dict(color='violet'), showlegend=False), row=1, col=3)
fig.add_trace(go.Scatter(x=vs_new_rsv[:,3], y=z, name='Sw=20%',
                         line=dict(color='red'), showlegend=False), row=1, col=3)
fig.add_trace(go.Scatter(x=vs_new_rsv[:,4], y=z, mode='lines', name='Sw=5%',
                         line=dict(color='yellow'), showlegend=False), row=1, col=3)
fig.add_trace(go.Scatter(x=vs_new_rsv[:,5], y=z, mode='lines', name='Sw=0%',
                         line=dict(color='black'), showlegend=False), row=1, col=3)

# Ajustando as propriedades dos eixos x
fig.update_xaxes(title_text="(m/s)", row=1, col=1,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text="(kg/m³)", row=1, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text="(m/s)", row=1, col=3,
                 title_font=dict(size=22), tickfont=dict(size=20))

# Ajustando as propriedades dos eixos y
fig.update_yaxes(title_text="Depth (m)", row=1, col=1, autorange="reversed",
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(autorange="reversed", row=1, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(autorange="reversed", row=1, col=3,
                 title_font=dict(size=22), tickfont=dict(size=20))

# Atualizando layout
fig.update_layout(height=600, showlegend=True, width=1200, title_text='Stages of mud filtrate invasion into a reservoir',
                  title_font=dict(size=24), title_x=0.5)

fig.update_annotations(font=dict(size=24))

# Colocando a legenda fora dos plots em um novo subplot
fig.update_layout(
    legend=dict(
        x=0.9,
        y=1,
        traceorder='normal',
        font=dict(size=20),
        bordercolor="Black"
    )
)

# Mostrando o plot
fig.show()


## <font color=orange> **Well tie**

In [197]:
# # seismic trace and wavelet from seismic model
trace_model = np.loadtxt(f'/content/drive/MyDrive/doutorado/mud_filtrate/dados/initial_trace_{len(vp[500:reservoir])}_{round(Por*100)}_20.txt')
wavelet_model = np.loadtxt('/content/drive/MyDrive/doutorado/mud_filtrate/dados/initial_wavelet_20.txt')

In [198]:
# impedance
ip = np.zeros((len(t), len(filtrate_sat)))
for i in range(len(filtrate_sat)):
    for j in range(len(t)):
        ip[j, i] = vp_new_rsv[j, i] * rho_new_rsv[j, i]

# reflectivity
R = np.zeros((len(t) - 1, len(filtrate_sat)))
for i in range(len(filtrate_sat)):
    for j in range(len(t) - 1):
        R[j, i] = (ip[j + 1, i] - ip[j, i]) / (ip[j + 1, i] + ip[j, i])

# convolution
synt = np.zeros((len(t), len(filtrate_sat)))
for i in range(len(filtrate_sat)):
    sp = np.convolve(R[:, i], wavelet_model)
    synt[:, i] = sp[:len(t)]

# creating a seismic trace with noise
synt_r = np.zeros((len(t), len(filtrate_sat)))
for i in range(n_z):
    h = np.random.rand(1) / 25
    synt_r[i] = synt[i] + h

In [199]:
# synthetic trace
fig = make_subplots(rows=1, cols=5, subplot_titles=('Impedance', 'Reflectivity', 'Wavelet', 'Synthetic 1', 'Seismic trace'))

fig.add_trace(go.Scatter(x=ip[:,0], y=t, name='Sw=100%',
                         line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=ip[:,1], y=t, name='Sw=60%',
                         line=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=ip[:,2], y=t, name='Sw=40%',
                         line=dict(color='violet')), row=1, col=1)
fig.add_trace(go.Scatter(x=ip[:,3], y=t, name='Sw=20%',
                         line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=ip[:,4], y=t, mode='lines', name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=1)
fig.add_trace(go.Scatter(x=ip[:,5], y=t, mode='lines', name='Sw=0%',
                         line=dict(color='black')), row=1, col=1)

fig.add_trace(go.Scatter(x=R[:,0], y=t, name='Sw=80%',
                         line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=R[:,1], y=t, name='Sw=60%',
                         line=dict(color='green')), row=1, col=2)
fig.add_trace(go.Scatter(x=R[:,2], y=t, name='Sw=40%',
                         line=dict(color='violet')), row=1, col=2)
fig.add_trace(go.Scatter(x=R[:,3], y=t, name='Sw=20%',
                         line=dict(color='red')), row=1, col=2)
fig.add_trace(go.Scatter(x=R[:,4], y=t, mode='lines', name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=2)
fig.add_trace(go.Scatter(x=R[:,5], y=t, mode='lines', name='Sw=0%',
                         line=dict(color='black')), row=1, col=2)

fig.add_trace(go.Scatter(x=wavelet_model, y=t, name='Sw=80%',
                         line=dict(color='blue')), row=1, col=3)

fig.add_trace(go.Scatter(x=synt_r[:,0], y=t, name='Sw=80%',
                         line=dict(color='blue')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt_r[:,1], y=t, name='Sw=60%',
                         line=dict(color='green')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt_r[:,2], y=t, name='Sw=40%',
                         line=dict(color='violet')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt_r[:,3], y=t, name='Sw=20%',
                         line=dict(color='red')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt_r[:,4], y=t, mode='lines', name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt_r[:,5], y=t, mode='lines', name='Sw=0%',
                         line=dict(color='black')), row=1, col=4)

fig.add_trace(go.Scatter(x=trace_model, y=t, name='Seismic model',
                         line=dict(color='blue')), row=1, col=5)

# Update xaxis properties
fig.update_xaxes(title_text='kg/(m²s)', row=1, col=1,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Unitless', row=1, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=3,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=4,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=5,
                 title_font=dict(size=22), tickfont=dict(size=20))

# Update yaxis properties
fig.update_yaxes(title_text="Time (s)", row=1, col=1, autorange="reversed",
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(autorange="reversed", row=1, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(autorange="reversed", row=1, col=3,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(autorange="reversed", row=1, col=4,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(autorange="reversed", row=1, col=5,
                 title_font=dict(size=22), tickfont=dict(size=20))

fig.update_layout(height=700, width=1200, showlegend=False, title_text='Synthetic generation',
                  title_font=dict(size=24))

fig.update_annotations(font=dict(size=24))

fig.show()

##<font color=orange> **Sparse-spike deconvolution**

In [200]:
def conjgrad(A, b, x=None):
    n = len(b)
    if x is None:
        x = np.zeros(n)
    r = b - A @ x
    p = r.copy()
    rsold = np.dot(r, r)

    for _ in range(n):
        Ap = A @ p
        alpha = rsold / np.dot(p, Ap)
        x = x + alpha * p
        r = r - alpha * Ap
        rsnew = np.dot(r, r)
        if np.sqrt(rsnew) < 1e-10:
            break
        p = r + (rsnew / rsold) * p
        rsold = rsnew

    return x

# Sparse-spike deconvolution function
def sparse_decon_l2(d, r, mu, iter_max):
    nr = len(r)
    nt, ntraces = d.shape

    # Increasing zeros to allow full convolution
    d_padded = np.vstack([d, np.zeros((nr - 1, ntraces))])

    # Creating convolution matrix
    R = toeplitz(np.concatenate([r, np.zeros(nt - 1)]), np.concatenate([[r[0]], np.zeros(nt - 1)]))

    # calculating the autocorrelation matrix
    R1 = R.T @ R

    # Creating the regularization matrix
    Q = mu * np.eye(nt)

    # scale value
    sc = 0.0001

    wave = np.zeros((nt, ntraces))

    for itrace in range(ntraces):
        s = d_padded[:, itrace]
        w = np.zeros(nt)

        for k in range(iter_max):
            g = R.T @ s
            Matrix = R1 + Q
            w = conjgrad(Matrix, g, w)

        wave[:, itrace] = w

    dp = np.apply_along_axis(lambda x: convolve(x, r, mode='full')[:nt], axis=0, arr=wave)

    return wave, dp

In [201]:
# estimating the wavelet - using synthetic trace and reflectivity
alpha = 0.01
synt2 = np.zeros((len(t), len(filtrate_sat)))
corrs = []
corrs2 = []
wavelet = []

for i in range(len(filtrate_sat)):
    w, _ = sparse_decon_l2(synt_r[:, i:i+1], R[:,i], alpha, 5)
    wavelet.append(w.squeeze())  # remove unnecessary dimension

    # synthetic trace 2 from the estimated wavelet
    synt2[:, i] = convolve(R[:,i], wavelet[i], mode='full')[:len(t)]
    # synt2[:, i] = synt2[:, i] / np.max(np.abs(synt2[:, i]))
    # synt[:, i] = synt[:, i] / np.max(np.abs(synt[:, i]))
    # wavelet[i] = wavelet[i] / np.max(np.abs(wavelet[i]))

    corr = np.corrcoef(synt[:, i], synt2[:, i])[0, 1]
    corrs.append(corr)

    corr2 = np.corrcoef(trace_model, synt2[:, i])[0, 1]
    corrs2.append(corr)

# convert list of numpy arrays to a single numpy array
wavelet = np.column_stack(wavelet)

In [202]:
# plot
fig = make_subplots(rows=1, cols=4, subplot_titles=('Reflectivities', 'Synthetic 1', 'Wavelets SS', 'Synthetic 2'))

fig.add_trace(go.Scatter(x=R[:,0], y=t, name='Sw=100%',
                         line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=R[:,1], y=t, name='Sw=60%',
                         line=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=R[:,2], y=t, name='Sw=40%',
                         line=dict(color='violet')), row=1, col=1)
fig.add_trace(go.Scatter(x=R[:,3], y=t, name='Sw=20%',
                         line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=R[:,4], y=t, mode='lines', name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=1)
fig.add_trace(go.Scatter(x=R[:,5], y=t, mode='lines', name='Sw=0%',
                         line=dict(color='black')), row=1, col=1)

fig.add_trace(go.Scatter(x=synt_r[:,0], y=t, name='Sw=100%',
                         line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=synt_r[:,1], y=t, name='Sw=60%',
                         line=dict(color='green')), row=1, col=2)
fig.add_trace(go.Scatter(x=synt_r[:,2], y=t, name='Sw=40%',
                         line=dict(color='violet')), row=1, col=2)
fig.add_trace(go.Scatter(x=synt_r[:,3], y=t, name='Sw=20%',
                         line=dict(color='red')), row=1, col=2)
fig.add_trace(go.Scatter(x=synt_r[:,4], y=t, mode='lines', name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=2)
fig.add_trace(go.Scatter(x=synt_r[:,5], y=t, mode='lines', name='Sw=0%',
                         line=dict(color='black')), row=1, col=2)

fig.add_trace(go.Scatter(x=wavelet[:, 0], y=t, name='Sw=100%',
                         line=dict(color='blue')), row=1, col=3)
fig.add_trace(go.Scatter(x=wavelet[:, 1], y=t, name='Sw=60%',
                         line=dict(color='green')), row=1, col=3)
fig.add_trace(go.Scatter(x=wavelet[:, 2], y=t, name='Sw=40%',
                         line=dict(color='violet')), row=1, col=3)
fig.add_trace(go.Scatter(x=wavelet[:, 3], y=t, name='Sw=20%',
                         line=dict(color='red')), row=1, col=3)
fig.add_trace(go.Scatter(x=wavelet[:, 4], y=t, name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=3)
fig.add_trace(go.Scatter(x=wavelet[:, 5], y=t, name='Sw=0%',
                         line=dict(color='black')), row=1, col=3)

fig.add_trace(go.Scatter(x=synt2[:,0], y=t, name='Sw=80%',
                         line=dict(color='blue')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt2[:,1], y=t, name='Sw=60%',
                         line=dict(color='green')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt2[:,2], y=t, name='Sw=40%',
                         line=dict(color='violet')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt2[:,3], y=t, name='Sw=20%',
                         line=dict(color='red')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt2[:,4], y=t, mode='lines', name='Sw=5%',
                         line=dict(color='yellow')), row=1, col=4)
fig.add_trace(go.Scatter(x=synt2[:,5], y=t, mode='lines', name='Sw=0%',
                         line=dict(color='black')), row=1, col=4)

# xaxis properties
fig.update_xaxes(title_text='Unitless', row=1, col=1,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=3,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Amplitude (a.u.)', row=1, col=4,
                 title_font=dict(size=22), tickfont=dict(size=20))

# yaxis properties
fig.update_yaxes(title_text="Time (s)", row=1, col=1, autorange="reversed",
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(autorange="reversed", row=1, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(autorange="reversed", row=1, col=3,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(autorange="reversed", row=1, col=4,
                 title_font=dict(size=22), tickfont=dict(size=20))

fig.update_layout(height=700, width=1200, showlegend=False, title_text='Second synthetic generation ',
                  title_font=dict(size=24))

fig.update_annotations(font=dict(size=24))

fig.show()

In [203]:
# correlation plots
fig = make_subplots(rows=3, cols=2,
                    subplot_titles=('0% of invasion', '40% of invasion',
                                    '5% of invasion','60% of invasion',
                                    '20% of invasion', '100% of invasion'))

fig.add_trace(go.Scatter(x=t, y=trace_model, name='Trace model',
                         line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=synt2[:,0], name='Sw=100%',
                         line=dict(color='red')), row=1, col=1)

fig.add_trace(go.Scatter(x=t, y=trace_model, name='Trace model',
                         line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=synt2[:,1], name='Sw=60%',
                         line=dict(color='red')), row=2, col=1)

fig.add_trace(go.Scatter(x=t, y=trace_model, name='Trace model',
                         line=dict(color='blue')), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=synt2[:,2], name='Sw=40%',
                         line=dict(color='red')), row=3, col=1)

fig.add_trace(go.Scatter(x=t, y=trace_model, name='Trace model',
                         line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=t, y=synt2[:,3], name='Sw=20%',
                         line=dict(color='red')), row=1, col=2)

fig.add_trace(go.Scatter(x=t, y=trace_model, name='Trace model',
                         line=dict(color='blue')), row=2, col=2)
fig.add_trace(go.Scatter(x=t, y=synt2[:,4], mode='lines', name='Sw=5%',
                         line=dict(color='red')), row=2, col=2)

fig.add_trace(go.Scatter(x=t, y=trace_model, name='Trace model',
                         line=dict(color='blue')), row=3, col=2)
fig.add_trace(go.Scatter(x=t, y=synt2[:,5], mode='lines', name='Sw=0%',
                         line=dict(color='red')), row=3, col=2)

# xaxis properties
fig.update_xaxes(row=1, col=1,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(row=2, col=1,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(row=3, col=1,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(row=1, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(row=2, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_xaxes(title_text='Time (s)', row=3, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))


# yaxis properties
fig.update_yaxes(title_text="Amp. (a.u.)", autorange="reversed", row=1, col=1,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(title_text="Amp. (a.u.)", autorange="reversed", row=2, col=1,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(title_text="Amp. (a.u.)", autorange="reversed", row=3, col=1,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(title_text="Amp. (a.u.)", autorange="reversed", row=1, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(title_text="Amp. (a.u.)", autorange="reversed", row=2, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))
fig.update_yaxes(title_text="Amp. (a.u.)", autorange="reversed", row=3, col=2,
                 title_font=dict(size=22), tickfont=dict(size=20))

# adds the correlation value as an annotation to the subplot
fig.add_annotation(x=0.5, y=0.95, xref='paper', yref='paper',
                   text=f'r: {corrs2[0]:.6f}', showarrow=False, row=1, col=1, font=dict(size=20))
fig.add_annotation(x=0.5, y=0.95, xref='paper', yref='paper',
                   text=f'r: {corrs2[1]:.6f}', showarrow=False, row=2, col=1, font=dict(size=20))
fig.add_annotation(x=0.5, y=0.95, xref='paper', yref='paper',
                   text=f'r: {corrs2[2]:.6f}', showarrow=False, row=3, col=1, font=dict(size=20))
fig.add_annotation(x=0.5, y=0.95, xref='paper', yref='paper',
                   text=f'r: {corrs2[3]:.6f}', showarrow=False, row=1, col=2, font=dict(size=20))
fig.add_annotation(x=0.5, y=0.95, xref='paper', yref='paper',
                   text=f'r: {corrs2[4]:.6f}', showarrow=False, row=2, col=2, font=dict(size=20))
fig.add_annotation(x=0.5, y=0.95, xref='paper', yref='paper',
                   text=f'r: {corrs2[5]:.6f}', showarrow=False, row=3, col=2, font=dict(size=20))

fig.update_layout(height=1000, width=1200, showlegend=False, title_text=f'Pearson correlation coefficient - Thickness: {len(vp[500:reservoir])}m, Porosity:{round(Por*100)}%, Frequency: 20Hz',
                  title_font=dict(size=24))

fig.update_annotations(font=dict(size=24))

fig.show()

In [204]:
np.savetxt(f'/content/drive/MyDrive/doutorado/mud_filtrate/dados/corr_{len(vp[500:reservoir])}_{round(Por*100)}_20.txt', corrs2)